<a href="https://colab.research.google.com/github/issacali/Case_Study_Kashti/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import tensorflow and checking the version
import tensorflow as tf

In [4]:
print(tf.__version__)

2.8.0


In [5]:
# install kaggle 
!pip install kaggle

In [12]:
!pwd

/content


In [14]:
!mkdir -p ~/.kaggle
!cp '/content/drive/My Drive/kaggle.json' ~/.kaggle
# checking permission 
!chmod 600  ~/.kaggle/kaggle.json

In [16]:
# now downloading data set let go 
!kaggle competitions download -c digit-recognizer

 98% 15.0M/15.3M [00:00<00:00, 153MB/s]
100% 15.3M/15.3M [00:00<00:00, 153MB/s]


In [18]:
# unzip files 
!unzip "digit-recognizer.zip"

# !unzip "train.csv.zip"
# !unzip "test.csv.zip"

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [19]:
!unzip "train.csv.zip"
!unzip "test.csv.zip"

unzip:  cannot find or open train.csv.zip, train.csv.zip.zip or train.csv.zip.ZIP.
unzip:  cannot find or open test.csv.zip, test.csv.zip.zip or test.csv.zip.ZIP.


In [20]:
!rm -rf test.csv.zip
!rm -rf test.csv.zip

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
import tensorflow as tf

from subprocess import check_output
# print(check_output(["1s","../content"]).decode("utf8"))

In [26]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [27]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [33]:
train_y=train["label"].astype("float32")
train_x=train.drop(["label"],axis=1).astype("int32")
test_x=test.astype("float32")

train_x.shape,train_y.shape,test_x.shape

(42000, 784)

In [34]:
train_x=train_x.values.reshape(-1,28,28,1)
train_x=train_x/255.0
tesxt_x=test_x.values.reshape(-1,28,28,1)
test_x=test_x/255.0
train_x.shape,test_x.shape

((42000, 28, 28, 1), (28000, 784))

In [35]:
# performing one hot encoding 
train_y=tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [36]:
print(train["label"].head())

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64


In [46]:
# defining the model for analysis 
model=tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=(28,28,1)),
  tf.keras.layers.Conv2D(32,(3,3),activation="relu"),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation="relu",padding="Same"),
  tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2),strides=(2,2),padding='valid'),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256,activation="relu"),
  tf.keras.layers.Dense(256,activation="relu"),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10,activation="softmax")

])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                  

In [47]:
# Defining the callback function to 
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get("accureacy")>0.999):
      print(" reached to 99.9% accuracy so stop the training")
      self.model.stop_training=True
callbaks=myCallback()

In [52]:
# Defining the callback function to 
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get("accureacy")>0.999):
      print(" reached to 99.9% accuracy so stop the training")
      self.model.stop_training=True
callbaks=myCallback()

# compiling and model taining with batch size=50, epochs=20,and optimizer= adam

Optimizer=tf.keras.optimizers.Adam(
    
           learning_rate=0.0005,
           beta_1=8.9,
           beta_2=0.999,
           epsilon=1e-07,
           name="Adam"
)
                      
model.compile(optimizer=Optimizer,loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(train_x,train_y,batch_size=50,epochs=20,callbacks=[callbaks])

Epoch 1/20
840/840 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0997

TypeError: ignored

In [54]:
results= model.predict(test_x)

# select the index with the max probabilty and lest check how it comes out
results =np.argmax(results,axix=1)
results= pd.Series(results,name="Label")

ValueError: ignored

In [55]:
submission= pd.concat([pd.Series(range(1,28001),name="ImageId"),results],axis=1)
submission.to_csv("submission.csv",index=False)

NameError: ignored

In [56]:

#Hey its time to take it to kaggle submission time hurry 
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/kaggle/cli.py", line 67, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 562, in competition_submit_cli
    competition, quiet)
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 513, in competition_submit
    content_length=os.path.getsize(file_name),
  File "/usr/lib/python3.7/genericpath.py", line 50, in getsize
    return os.stat(filename).st_size
FileNotFoundError: [Errno 2] No such file or directory: 'submission.csv'
